In [2]:
import numpy as np
import pandas as pd
import csv

n = 0
data = []
with open('data/vacs_train.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=';')
    for row in reader:
        #print(row['name'], row['description'])
        #print(row)
        n += 1
        data.append(row)
df = pd.DataFrame(data)
for c in ['company.id','salary_from']:
    df[c] = pd.to_numeric(df[c])
df_filtered = df[(df['salary_from'] >= 20000) & (df['salary_from'] <= 100000)]


In [3]:
#df_filtered = df_filtered.iloc[:10000]

df_filtered.reset_index(drop=True, inplace=True)

data = pd.DataFrame()
data['text'] = df_filtered['name'] + '. ' + df_filtered['description'] + '. Ключевые навыки: ' + df_filtered['key_skills'] + '. Специализации: ' + df_filtered['specializations.names'] + '. Тип занятости: ' + df_filtered['employment.name'] + '. График: ' + df_filtered['schedule.name'] + '. Требуемый опыт работы: ' + df_filtered['experience.name'] + '. Компания ' + df_filtered['company']

y = np.zeros(df_filtered.index.max() + 1)
bins = np.array([0, 25000.0, 30000.0, 35000.0, 40000.0, 45000.0, 60000.0, 400000.0])
for i, val in df_filtered['salary_from'].iteritems():
    j = 0
    for k in range(0, len(bins)):
        if val < bins[k]:
            j = k
            break
    y[i] = (k - 1)
y = pd.Series(y)
data['label'] = y

In [5]:
from sklearn.model_selection import train_test_split

train_df,  eval_df = train_test_split(data, test_size=0.3, random_state=42)

In [6]:
eval_df['label'].value_counts()

5.0    5099
2.0    5065
4.0    4259
1.0    4227
6.0    4013
3.0    3349
0.0    2459
Name: label, dtype: int64

In [ ]:
from simpletransformers.classification import ClassificationModel

import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# Create a ClassificationModel
model = ClassificationModel('bert', 'DeepPavlov/rubert-base-cased', num_labels=7, args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'train_batch_size':16, 'eval_batch_size':16, 'epochs': 1})

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

print(result)
#print(model_outputs)

